# Метод Ньютона

Метод Ньютона -- численный метод решения уравнения
$$f(x) = 0, $$

где отображение $f$:
$$f: \, \mathbb R^n \to \mathbb R^n,$$
$$f_i: \, \mathbb R^n \to \mathbb R,$$
$$f(x) = (f_i(x)).$$

Метод Ньютона:
$$f(x^k) + J(x^k) (x^{k + 1} - x^k) = 0,$$
$$x = \lim_{k \to \infty} x^k,$$

где $J$ -- матрица Якоби отображения $f$:
$$J(x) = \left( \partial f_i (x) \over \partial x_j \right).$$

Алгоритм метода Ньютона:
$$J(x^k) \, \delta^k = - f(x^k),$$
$$x^{k + 1} = x^k + \delta^k.$$

Аппроксимация элемента матрицы Якоби конечной разностью вперёд:
$${\partial f_i (x) \over \partial x_j}
\approx {f_i (x + n_j \, dx_{ij}) - f_i (x) \over dx_{ij}},$$

где $n_j$ -- единичный вектор, j-тая компонента которого равна 1:
$$n_i \in \mathbb R^n: \, (j = i \rightarrow {n_i}_j = 1)
\, \land (j \neq i \rightarrow {n_i}_j = 0),$$

$dx_{ij}$ -- шаг конечной разности вперед у элемента матрицы.

# Реализация и проверка

Система нелинейных уравнений и ее точное решение:
$$(i = 1) \quad (3 + 2x_i)\,x_i - 2x_{i + 1} = 3,$$
$$(\forall i \in \mathbb N[2, n - 1]) \quad (3 + 2x_i)\,x_i - x_{i - 1} - 2x_{i + 1} = 2,$$
$$(i = n) \quad (3 + 2x_i)\,x_i - x_{i - 1} = 4.$$
$$\Leftrightarrow$$
$$(\forall i \in \mathbb N[1, n]) \quad x_i = 1.$$

Эквивалентное уравнение:
$$f(x) = 0,$$

где отображение $f$:
$$(i = 1) \quad f_i(x) = (3 + 2x_i)\,x_i - 2x_{i + 1} - 3,$$
$$(\forall i \in \mathbb N[2, n - 1]) \quad f_i(x) = (3 + 2x_i)\,x_i - x_{i - 1} - 2x_{i + 1} - 2,$$
$$(i = n) \quad f_i(x) = (3 + 2x_i)\,x_i - x_{i - 1} - 4.$$

In [3]:
import numpy as np


def newton_method(f, x0, dx, n_iteration):
    n = len(x0)
    unit_vector = np.eye(n)
    J = lambda x: np.array([[(f(x + unit_vector[j] * dx[i, j])[i] - f(x)[i]) / dx[i, j]
                             for j in range(n)] for i in range(n)])

    x = x0
    for i in range(n_iteration):
        sigma = np.linalg.solve(J(x), - f(x))
        x = sigma + x

    return x


f = lambda x: np.array(
    [(3 + 2 * x[i]) * x[i]            - 2 * x[i + 1] - 3 for i in [0]] +
    [(3 + 2 * x[i]) * x[i] - x[i - 1] - 2 * x[i + 1] - 2 for i in range(1, n - 2 + 1)] +
    [(3 + 2 * x[i]) * x[i] - x[i - 1]                - 4 for i in [n - 1]]
)

n = 10
x0 = np.zeros(n)
dx = np.ones((n, n)) * 1e-8

x = newton_method(f, x0, dx, n_iteration=10)

x

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Численное решение сходится с точными решением.

# Векторизация

Вывод векторизации:
$$\left[ f_i (x + n_j \, dx_{ij}) - f_i (x) \over dx_{ij} \right]
= {[f_i (x + n_j \, dx_{ij})] - [f_i (x)]^T \over [dx_{ij}]}$$
$$= {\bar f(x + [n_j \, dx_{ij}]) - f(x)^T \over dx}
= {\bar f(x + [n \, dx_i]) - f(x)^T \over dx}.$$
$$dx = (dx_{ij}).$$
$$\bar f: \, \mathbb R^{n \times n} \to \mathbb R^n,$$
$$\bar f(x) = (f_i(x_i)).$$
$$x = (x_i),$$
$$x_i = (x_{ij})_j,$$
$$x = (x_{ij}).$$

Ниже проверка формулы $[f_i (x + n_j \, dx_{ij})] = \bar f(x + [n \, dx_i])$ с помощью Numpy.

Отображение $f$:
$$f(x) = (f_i(x)).$$
$$(i = 1) \quad f_i(x) = (3 + 2x_i)\,x_i - 2x_{i + 1} - 3,$$
$$(\forall i \in \mathbb N[2, n - 1]) \quad f_i(x) = (3 + 2x_i)\,x_i - x_{i - 1} - 2x_{i + 1} - 2,$$
$$(i = n) \quad f_i(x) = (3 + 2x_i)\,x_i - x_{i - 1} - 4.$$

In [4]:
bar_f = lambda x: np.array(
    [(3 + 2 * x[i, i]) * x[i, i]               - 2 * x[i, i + 1] - 3 for i in [0]] +
    [(3 + 2 * x[i, i]) * x[i, i] - x[i, i - 1] - 2 * x[i, i + 1] - 2 for i in range(1, n - 2 + 1)] +
    [(3 + 2 * x[i, i]) * x[i, i] - x[i, i - 1]                   - 4 for i in [n - 1]]
)

bar_f_vec = np.vectorize(bar_f, signature='(n,n)->(n)')

In [6]:
n = 5
x = np.arange(n)
dx = (np.arange(n * n).reshape(n, n) + 1) * 1e-1
unit_vector = np.eye(n)

$[f_i (x + n_j \, dx_{ij})] =$

In [28]:
np.array([[f(x + unit_vector[j] * dx[i, j])[i] for j in range(n)] for i in range(n)])

array([[-4.68, -5.4 , -5.  , -5.  , -5.  ],
       [-1.6 ,  4.88, -2.6 , -1.  , -1.  ],
       [ 5.  ,  3.8 , 22.68,  2.2 ,  5.  ],
       [15.  , 15.  , 13.2 , 50.72, 11.  ],
       [37.  , 37.  , 37.  , 34.6 , 97.  ]])

In [26]:
%timeit np.array([[f(x + unit_vector[j] * dx[i, j])[i] for j in range(n)] for i in range(n)])

554 µs ± 82.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


$\bar f(x + [n \, dx_i]) =$

In [30]:
bar_f_vec(x + np.array([unit_vector * dx[i] for i in range(n)]))

array([[-4.68,  0.48,  8.48, 21.32, 47.  ],
       [-2.48,  4.88, 15.08, 30.12, 58.  ],
       [ 0.72, 10.28, 22.68, 39.92, 70.  ],
       [ 4.92, 16.68, 31.28, 50.72, 83.  ],
       [10.12, 24.08, 40.88, 62.52, 97.  ]])

In [31]:
%timeit bar_f_vec(x + np.array([unit_vector * dx[i] for i in range(n)]))

352 µs ± 59.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Сходятся только диагональные элементы. Формула $[f_i (x + n_j \, dx_{ij})] = \bar f(x + [n \, dx_i])$ неверная.

помогите плиз векторизовать вычисление матрицы Якоби с конечной разностью вперед
$$\left[ f_i (x + n_j \, dx_{ij}) - f_i (x) \over dx_{ij} \right],$$

где $n_j$ -- единичный вектор, j-тая компонента которого равна 1;
$dx_{ij}$ -- шаг конечной разности вперед у элемента матрицы.